In [8]:

import qiskit
print(qiskit.__version__) 

1.3.1


In [9]:
from qiskit_algorithms.minimum_eigensolvers import QAOA                             
from qiskit_algorithms.optimizers import COBYLA
from qiskit_aer.primitives import Sampler
from qiskit_finance.applications.optimization import PortfolioOptimization
from qiskit_finance.data_providers import RandomDataProvider,YahooDataProvider
from qiskit_optimization.algorithms import MinimumEigenOptimizer
import datetime

$$
\begin{aligned}
\min_{x \in \{0, 1\}^n}\quad& q x^T \Sigma x - \mu^T x\\
\text{s.t.}\quad& 1^T x = B
\end{aligned}
$$

定义基本的参数，构造数据。

In [10]:
num_assets = 4
data = RandomDataProvider(
    tickers=[f"TICKERS{i}" for i in range(num_assets)],
    start=datetime.datetime(2021, 11, 17),
    end=datetime.datetime(2022, 11, 17),
)
# data = YahooDataProvider(
#     tickers=["MSFT", "AAPL", "GOOG"],
#     start=datetime.datetime(2021, 1, 1),
#     end=datetime.datetime(2021, 12, 31),
# )
data.run()
mu = data.get_period_return_mean_vector()
sigma = data.get_period_return_covariance_matrix()
q = 0.5
B = num_assets // 2

定义投资组合优化问题。

In [11]:
problem = PortfolioOptimization(mu, sigma, q, B)

将原问题转化为一个二次规划问题。

In [12]:
quadratic_program = problem.to_quadratic_program()
print(quadratic_program)

minimize 0.007428141257455332*x_0^2 - 0.00014646452452063768*x_0*x_1 + 1.2694914454502858e-05*x_0*x_2 + 6.537730465747055e-05*x_0*x_3 + 0.00011451021949265586*x_1^2 - 4.273910587657039e-06*x_1*x_2 - 1.3178550991620608e-05*x_1*x_3 + 4.8795161029921835e-05*x_2^2 - 9.093994948981333e-06*x_2*x_3 + 8.490486718410337e-05*x_3^2 + 0.005231170899957208*x_0 - 0.00034547772956898336*x_1 + 6.590943566140697e-05*x_2 - 0.0005596412317659577*x_3 (4 variables, 1 constraints, 'Portfolio optimization')


利用QAOA求解，得到投资方案。

In [13]:
result = MinimumEigenOptimizer(QAOA(Sampler(), COBYLA())).solve(
    quadratic_program
)
print(result)

fval=-0.0007188824256498024, x_0=0.0, x_1=1.0, x_2=0.0, x_3=1.0, status=SUCCESS


输出全部的可能方案，得到满足约束方案的概率大。

In [14]:
print("投资方案\t\t概率\t目标函数值")
for (
    k,
    v,
) in sorted(
    result.min_eigen_solver_result.eigenstate.binary_probabilities().items(),
    key=lambda x: x[1],
    reverse=True,
):
    x = [int(i) for i in list(reversed(k))]
    print(
        f"{x}\t{v:.4f}\t{problem.to_quadratic_program().objective.evaluate(x):.6f}"
    )

投资方案		概率	目标函数值
[1, 1, 0, 0]	0.1211	0.012282
[1, 0, 0, 1]	0.1152	0.012250
[0, 1, 0, 1]	0.1094	-0.000719
[1, 0, 1, 0]	0.1045	0.012787
[0, 0, 1, 1]	0.0947	-0.000369
[0, 1, 1, 0]	0.0938	-0.000121
[1, 1, 0, 1]	0.0508	0.011859
[1, 0, 1, 1]	0.0508	0.012368
[0, 0, 1, 0]	0.0498	0.000115
[0, 0, 0, 1]	0.0439	-0.000475
[1, 1, 1, 0]	0.0439	0.012405
[0, 1, 1, 1]	0.0430	-0.000618
[0, 1, 0, 0]	0.0420	-0.000231
[1, 0, 0, 0]	0.0371	0.012659
